# Diabetes Machine Learning Group Project

Data Tables from 2017-18 from NHANES
- Diabetes (DIQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DIQ_J.htm
- Health Insurance (HIQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/HIQ_J.htm
- Food Security (FSQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/FSQ_J.htm
- Income (INQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/INQ_J.htm
- Blood Pressure (BPX_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BPX_J.htm
- Body Measures (BMX_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BMX_J.htm
- Alcohol Use (ALQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/ALQ_J.htm
- Physical Activity (PAQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/PAQ_J.htm
- Weight History (WHQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/WHQ_J.htm
- Diet Behavior and Nutrition (DBQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DBQ_J.htm

8,897 Original Respondents

STEPS
1. DONE: Format and subset for columns and respondents of interest
2. Scale and normalize values
3. Run Principal Component Analysis (PCA)
4. Examine which components correlate with which variables
5. Generate elbow plot to identify ideal number of clusters
6. Run K-means clustering using elbow plot deemed number of PCs
7. Identify which variables are most associated with clustering 

## Merging

In [63]:
# Load packages
import pandas as pd

In [64]:
# Load in tables
diq = pd.read_csv("./resources/DIQ_J_2017-18.csv", index_col = 0)
fsq = pd.read_csv("./resources/FSQ_J_2017-18.csv", index_col = 0)
inq = pd.read_csv("./resources/INQ_J_2017-18.csv", index_col = 0)
hiq = pd.read_csv("./resources/HIQ_J_2017-18.csv", index_col = 0)
bpx = pd.read_csv("./resources/BPX_J_2017-18.csv", index_col = 0)
bmx = pd.read_csv("./resources/BMX_J_2017-18.csv", index_col = 0)
alq = pd.read_csv("./resources/ALQ_J_2017-18.csv", index_col = 0)
paq = pd.read_csv("./resources/PAQ_J_2017-18.csv", index_col = 0)
whq = pd.read_csv("./resources/WHQ_J_2017-18.csv", index_col = 0)
dbq = pd.read_csv("./resources/DBQ_J_2017-18.csv", index_col = 0)

### Format health insurance

In [65]:
# Rename column to intuitive name
hiq = hiq.rename({"HIQ011": "health_insurance"}, axis = 1)

# Subset columns of interest
hiq = hiq[["SEQN", "health_insurance"]]

# Subset for yes and no
hiq = hiq.loc[hiq['health_insurance'].isin([1,2])]

### Format alcohol

In [66]:
# Rename column to intuitive name
alq = alq.rename({"ALQ121": "12mo_alcohol",
                  "ALQ151": "many_drinks_regularly"}, axis = 1)

# Subset columns of interest
alq = alq[["SEQN", "12mo_alcohol", "many_drinks_regularly"]]

# Subset for yes and no
alq = alq.loc[alq['12mo_alcohol'].isin([0,1,2,3,4,5,6,7,8,9,10])]
alq = alq.loc[alq['many_drinks_regularly'].isin([1,2])]

### Format physical activity

In [67]:
# Rename column to intuitive name
paq = paq.rename({"PAD680": "minutes_sedentary",
                  "PAQ620": "mod_work_activity",
                  "PAQ605": "vig_work_activity",
                  "PAQ635": "walk_bicycle",
                  "PAQ650": "vig_rec_activity",
                  "PAQ665": "mod_rec_activity"}, axis = 1)

# Subset columns of interest
paq = paq[["SEQN", "minutes_sedentary", "mod_work_activity",
          "vig_work_activity", "walk_bicycle", "vig_rec_activity",
          "mod_rec_activity"]]

# Subset for yes and no
paq = paq.loc[paq['minutes_sedentary'].isin(range(1321))]
paq = paq.loc[paq['mod_work_activity'].isin([1,2])]
paq = paq.loc[paq['vig_work_activity'].isin([1,2])]
paq = paq.loc[paq['walk_bicycle'].isin([1,2])]
paq = paq.loc[paq['vig_rec_activity'].isin([1,2])]
paq = paq.loc[paq['mod_rec_activity'].isin([1,2])]

### Format diet behavior and nutrition

In [68]:
# Rename column to intuitive name
dbq = dbq.rename({"DBQ700": "diet_health",
                  "DBD895": "week_meals_not_home",
                  "DBD910": "month_frozen_food"}, axis = 1)

# Subset columns of interest
dbq = dbq[["SEQN", "diet_health", "week_meals_not_home", "month_frozen_food"]]

# Subset for yes and no
dbq = dbq.loc[dbq['diet_health'].isin([1,2,3,4,5])]
dbq = dbq.loc[dbq['week_meals_not_home'].isin(range(91))]
dbq = dbq.loc[dbq['month_frozen_food'].isin(range(91))]

### Format weight history

In [69]:
# Rename column to intuitive name
whq = whq.rename({"WHD010": "height_in",
                  "WHQ225": "10lb_losses"}, axis = 1)

# Subset columns of interest
whq = whq[["SEQN", "height_in", "10lb_losses"]]

# Subset for yes and no
whq = whq.loc[whq['height_in'].isin(range(49,83))]
whq = whq.loc[whq['10lb_losses'].isin([1,2,3,4,5])]

### Format body measures

In [70]:
# Rename column to intuitive name
bmx = bmx.rename({"BMXBMI": "bmi", "BMXWT": "weight"}, axis = 1)

# Subset columns of interest
bmx = bmx[["SEQN", "bmi", "weight"]]

# Subset for non NAs
bmx = bmx.dropna(axis = 0)

### Format blood pressure

In [71]:
# Rename column to intuitive name
bpx = bpx.rename({"BPXPULS": "pulse"}, axis = 1)

# Subset columns of interest
bpx = bpx[["SEQN", "pulse"]]

# Subset for yes and no
bpx = bpx.loc[bpx['pulse'].isin([1,2])]

### Format income

In [72]:
# Rename column to intuitive name
inq = inq.rename({"IND235": "monthly_income", "INQ300": "family_savings",
                  "INQ320": "grocery_transport"}, axis = 1)

# Subset columns of interest
inq = inq[["SEQN", "monthly_income", "family_savings", "grocery_transport"]]

# Subset for yes and no
inq = inq.loc[inq['family_savings'].isin([1,2])]
inq = inq.loc[inq['grocery_transport'].isin([1,2,3,4,5,6,7,8,9])]
inq = inq.loc[inq['monthly_income'].isin([1,2,3,4,5,6,7,8,9,10,11,12])]

### Format food security

In [73]:
# Rename column to intuitive name
fsq = fsq.rename({"FSD032A": "worry_food", "FSD032B": "food_not_last",
                  "FSD032C": "balanced_meals", "FSDHH": "food_security",
                  "FSD151": "emergency_food", "FSQ165": "food_stamp",}, axis = 1)

# Subset columns of interest
fsq = fsq[["SEQN", "worry_food", "food_not_last", "balanced_meals",
           "food_security", "emergency_food", "food_stamp"]]

# Subset for yes and no
fsq = fsq.loc[fsq['worry_food'].isin([1,2,3])]
fsq = fsq.loc[fsq['food_not_last'].isin([1,2,3])]
fsq = fsq.loc[fsq['balanced_meals'].isin([1,2,3])]
fsq = fsq.loc[fsq['food_security'].isin([1,2,3,4])]
fsq = fsq.loc[fsq['emergency_food'].isin([1,2])]
fsq = fsq.loc[fsq['food_stamp'].isin([1,2])]

### Format diabetes

In [74]:
# Rename column to intuitive name
diq = diq.rename({"DIQ010": "diabetes", "DIQ050": "insulin"}, axis = 1)

# Subset columns of interest
diq = diq[["SEQN", "diabetes", "insulin"]]

# Subset for yes and no
diq = diq.loc[diq['diabetes'].isin([1,2,3])]
diq = diq.loc[diq['insulin'].isin([1,2])]

In [75]:
# Generate merged df of everything
merged = pd.merge(diq, fsq, on = "SEQN")
merged = pd.merge(merged, inq, on = "SEQN")
merged = pd.merge(merged, hiq, on = "SEQN")
merged = pd.merge(merged, bpx, on = "SEQN")
merged = pd.merge(merged, bmx, on = "SEQN")
merged = pd.merge(merged, alq, on = "SEQN")
merged = pd.merge(merged, paq, on = "SEQN")
merged = pd.merge(merged, whq, on = "SEQN")
merged = pd.merge(merged, dbq, on = "SEQN")

In [76]:
# Export merged dataframe
merged.to_csv("./resources/merged.csv")

In [77]:
merged.columns

Index(['SEQN', 'diabetes', 'insulin', 'worry_food', 'food_not_last',
       'balanced_meals', 'food_security', 'emergency_food', 'food_stamp',
       'monthly_income', 'family_savings', 'grocery_transport',
       'health_insurance', 'pulse', 'bmi', 'weight', '12mo_alcohol',
       'many_drinks_regularly', 'minutes_sedentary', 'mod_work_activity',
       'vig_work_activity', 'walk_bicycle', 'vig_rec_activity',
       'mod_rec_activity', 'height_in', '10lb_losses', 'diet_health',
       'week_meals_not_home', 'month_frozen_food'],
      dtype='object')

In [78]:
merged.shape

(3441, 29)